In [6]:

import pandas as pd

# Load the data from the file
file_path = "C:\\Users\\szhang\\Downloads\\00001_12876_Dept-Shift-Counts_v1.0_11-13-2019 1_rs.txt"
file_path = 'C:\\Users\\szhang\\Downloads\\test NYU.txt'
data = pd.read_csv(file_path, delimiter=',', skipinitialspace=True)

# Drop unnecessary columns
data = data.drop(columns=['Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9', 'LINE'])

# Extract the hour from 'Sched Start Time'
data['Hour'] = data['Sched Start Time'].apply(lambda x: int(str(x).split()[-1][:2]))

# Group by 'Week Day' and 'Hour' to count the number of employees
employee_counts = data.groupby(['Week Day', 'Hour']).size().reset_index(name='Employee Count')

# Group by 'Week Day' to count the total number of employees for each day
total_employee_counts = data.groupby('Week Day').size().reset_index(name='Total Employee Count')

# Merge the two DataFrames to compare peak hour counts with total counts
merged_data = employee_counts.merge(total_employee_counts, on='Week Day')

# Identify peak hours (hours with the highest employee count for each day)
peak_hours = merged_data.loc[merged_data.groupby('Week Day')['Employee Count'].idxmax()]

peak_hours


,Week Day,Hour,Employee Count,Total Employee Count
0,Mon,10,1,11
11,Sun,11,1,5
20,Tues,70,2,10
